In [88]:
import numpy as np
from scipy.fft import fft2, ifft2
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Step 1: Initialize spatial domain and parameters
L = 20  # Domain size (-10 to 10 for x and y)
n = 64  # Grid size
x = np.linspace(-L/2, L/2, n, endpoint=False)
y = np.linspace(-L/2, L/2, n, endpoint=False)
X, Y = np.meshgrid(x, y)


# Parameters
beta = 1
D1, D2 = 0.1, 0.1
tspan = np.arange(0, 4.5, 0.5)  # Time span (0 to 4 in steps of 0.5)

# Step 2: Define initial conditions (spiral pattern)
m = 1  # Number of spirals
u = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m * np.angle(X + 1j*Y) - np.sqrt(X**2 + Y**2))
v = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m * np.angle(X + 1j*Y) - np.sqrt(X**2 + Y**2))

# Fourier transform of initial conditions
ut = fft2(u)
vt = fft2(v)

# Stack initial condition in Fourier domain
initial_condition = np.hstack([ut.flatten(), vt.flatten()])

"""
# Step 3: Define spectral k values for periodic boundaries
kx = (2 * np.pi / L) * np.fft.fftfreq(n, d=1/n)
ky = (2 * np.pi / L) * np.fft.fftfreq(n, d=1/n)
KX, KY = np.meshgrid(kx, ky)
k_mesh = KX**2 + KY**2
k_mesh[0, 0] = 1e-6  # Avoid division by zero
"""
# Define spectral k values
kx = (2 * np.pi / L) * np.concatenate((np.arange(0, n/2), np.arange(-n/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / L) * np.concatenate((np.arange(0, n/2), np.arange(-n/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

# Step 4: Define reaction-diffusion system in Fourier domain
def reaction_diffusion_rhs(t, uv_flat, n, beta, D1, D2, K):
    # Unpack u and v in Fourier domain
    ut = uv_flat[:n*n].reshape((n, n))
    vt = uv_flat[n*n:].reshape((n, n))
    
    # Transform back to physical domain
    u = ifft2(ut)
    v = ifft2(vt)
    
    # Compute A^2 = U^2 + V^2
    A2 = u**2 + v**2
    
    # Reaction terms
    lambda_u = (1 - A2) * u + beta * A2 * v
    lambda_v = -beta * A2 * u + (1 - A2) * v
 
    # Fourier transform of reaction terms
    lambda_ut = fft2(lambda_u)
    lambda_vt = fft2(lambda_v)
    
    # Compute RHS in Fourier domain
    dut = -D1 * K * ut + lambda_ut
    dvt = -D2 * K * vt + lambda_vt
    
    # Flatten and stack
    return np.hstack([dut.flatten(), dvt.flatten()])

# Step 5: Solve the system using solve_ivp
sol = solve_ivp(
    reaction_diffusion_rhs,
    [tspan[0], tspan[-1]],
    initial_condition,
    t_eval=tspan,
    args=(n, beta, D1, D2, K),
    method='RK45'
)

# Step 6: Extract solution A1
A1 = sol.y
A1

array([[ 24.94003847+0.00000000e+00j,  12.73268299-9.84977910e-16j,
         -1.38095598+1.05760477e-15j, ..., -64.02389647-2.70260133e-14j,
        -67.76356741-3.61721539e-14j, -61.18058974-4.80159526e-14j],
       [-18.55666362-5.81663109e+01j, -42.51586944-4.69129224e+01j,
        -60.80795253-2.57480390e+01j, ..., -26.39439597+1.13082890e+02j,
          6.86544434+1.23000456e+02j,  41.4436393 +1.10055312e+02j],
       [-16.04755868+3.28279829e+01j, -22.03971648-4.57977740e+01j,
        -23.23089505-1.04141716e+02j, ..., -25.03391682-9.26527314e+01j,
        -29.2936105 -4.09594873e+01j, -31.3712619 +1.56986891e+01j],
       ...,
       [ 24.73021466-5.66774723e+02j,  34.94179045-3.31372917e+02j,
         38.82924248-4.97842318e+01j, ...,   4.99619196+6.02396295e+02j,
         -9.93322885+4.90736906e+02j, -25.6299042 +2.81792021e+02j],
       [ 25.33720124-3.61633792e+02j,  43.00958768-4.53711746e+02j,
         51.93221654-4.47841562e+02j, ..., -30.76392977+2.66442187e+02j,
       

In [109]:
# Fourier Transform

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.fft import fft2, ifft2

Lx = 20
Ly = 20
nx = 64
ny = 64
N = nx * ny
t = np.arange(0, 4.5, 0.5)  # time array

x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)

#Initial Conditions
m = 1 # number of spirals
beta = 1  # Reaction coefficient
D1 = D2 = 0.1  # Diffusion coefficients

r = np.sqrt(X**2 + Y**2)
theta = np.angle(X + 1j*Y)

u = np.tanh(r) * np.cos(m * theta - r)
v = np.tanh(r) * np.sin(m * theta - r)

uhat = fft2(u).reshape(N)
vhat = fft2(v).reshape(N)

uv = np.hstack([uhat, vhat])

kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

#Reaction-Diffusion
def rhs(t, uv, nx, ny, beta,  D1, D2, K):
    ut = uv[:nx * ny].reshape((nx , ny))
    vt = uv[nx * ny:].reshape((nx , ny))
    U = ifft2(ut)
    V = ifft2(vt)
    
    A = U*U + V*V
    
    dut = (- D1 * K * ut + fft2((1 - A) * U + beta * A * V)).reshape((N))
    dvt = (- D2 * K * vt + fft2(-beta * A * U + (1 - A) * V)).reshape((N))
    return np.hstack([(dut), (dvt)])

ftsol = solve_ivp(rhs, [t[0], t[-1]], uv, t_eval=t , args = (nx, ny, beta,  D1, D2, K) , method = 'RK45')
A1 = ftsol.y
A1

array([[ 24.94003847+0.00000000e+00j,  12.73268299-9.84977910e-16j,
         -1.38095598+1.05760477e-15j, ..., -64.02389647-2.70260133e-14j,
        -67.76356741-3.61721539e-14j, -61.18058974-4.80159526e-14j],
       [-18.55666362-5.81663109e+01j, -42.51586944-4.69129224e+01j,
        -60.80795253-2.57480390e+01j, ..., -26.39439597+1.13082890e+02j,
          6.86544434+1.23000456e+02j,  41.4436393 +1.10055312e+02j],
       [-16.04755868+3.28279829e+01j, -22.03971648-4.57977740e+01j,
        -23.23089505-1.04141716e+02j, ..., -25.03391682-9.26527314e+01j,
        -29.2936105 -4.09594873e+01j, -31.3712619 +1.56986891e+01j],
       ...,
       [ 24.73021466-5.66774723e+02j,  34.94179045-3.31372917e+02j,
         38.82924248-4.97842318e+01j, ...,   4.99619196+6.02396295e+02j,
         -9.93322885+4.90736906e+02j, -25.6299042 +2.81792021e+02j],
       [ 25.33720124-3.61633792e+02j,  43.00958768-4.53711746e+02j,
         51.93221654-4.47841562e+02j, ..., -30.76392977+2.66442187e+02j,
       

In [128]:
# ChebyChev version

import numpy as np
from numpy import *
import matplotlib.pyplot as plt


def cheb(N):
	if N==0: 
		D = 0.; x = 1.
	else:
		n = arange(0,N+1)
		x = cos(pi*n/N).reshape(N+1,1) 
		c = (hstack(( [2.], ones(N-1), [2.]))*(-1)**n).reshape(N+1,1)
		X = tile(x,(1,N+1))
		dX = X - X.T
		D = dot(c,1./c.T)/(dX+eye(N+1))
		D -= diag(sum(D.T,axis=0))
	return D, x.reshape(N+1)

N = 30

D , x = cheb(N)
D[N, :] = 0
D[0, :] = 0

Dxx = np.dot(D, D) / (20/2)**2

y = x
N2 = (N+1)* (N+1)
I = np.eye(len(Dxx))
L = np.kron(I,Dxx) +np.kron(Dxx,I) #2D Laplacian

X , Y = np.meshgrid(x,y)
X = X * (20/2)
Y = Y * (20/2)

m = 1 # number of spirals
beta = 1  # Reaction coefficient
D1 = D2 = 0.1  # Diffusion coefficients
t = np.arange(0, 4.5, 0.5)

u = np.tanh(np.sqrt(X**2 + Y**2)) * np.cos(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
v = np.tanh(np.sqrt(X**2 + Y**2)) * np.sin(m * np.angle(X + 1j * Y) - np.sqrt(X**2 + Y**2))
uv0 = np.hstack([u.flatten(), v.flatten()])  # Flatten and stack

def RD(t, uv, N, beta, D1, D2, L):
    N2 = (N + 1) ** 2
    uf = uv[:N2]
    vf = uv[N2:]
    u = uf.reshape((N + 1, N + 1))
    v = vf.reshape((N + 1, N + 1))

    # Compute A^2 = U^2 + V^2
    A = u**2 + v**2
    lam = 1 - A
    omega = -beta * A

    # Compute RHS
    rhs_u = D1 * L @ uf + lam.flatten() * uf - omega.flatten() * vf
    rhs_v = D2 * L @ vf + omega.flatten() * uf + lam.flatten() * vf
    return np.hstack([rhs_u, rhs_v])

chebsol = solve_ivp(RD, [t[0], t[-1]], uv0, t_eval=t , args = (N, beta,  D1, D2, L) , method = 'RK45')
A2 = chebsol.y